<a href="https://colab.research.google.com/github/atstuyuki/ScikitLearn/blob/main/mediapipe_throwing_pose_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [15]:
# ライブラリーのインストール
!pip install numpy==1.19.3
!pip install mediapipe

In [16]:
#手元の動画をgoogle colabにupload

from google.colab import files
import os
uploaded = files.upload()
#uploadedは辞書型なのでファイル名を取り出し、絶対パスを取得してfullpathという変数を当てる
for k in uploaded.keys():
  print('file name:',k)
fullpath=os.path.abspath(k)
print('fullpath:',fullpath)


Saving holistic.jpg to holistic.jpg
file name: holistic.jpg
fullpath: /content/holistic.jpg


In [17]:
#3次元座標の角度計算の定義
def calculate_3D_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #0がx座標　１がｙ座標　２がｚ座標
    #ベクトルを定義 長さと内積からコサインを計算
    vec_a=a-b
    vec_c=c-b
    len_vec_a=np.linalg.norm(vec_a)
    len_vec_c=np.linalg.norm(vec_c)
    inner_product=np.inner(vec_a, vec_c)
    cos=inner_product/(len_vec_a*len_vec_c)
    
    #角度（ラジアン）の計算
    rad=np.arccos(cos)
    
    #ラジアンから度数へ変換
    angle=np.rad2deg(rad)
        
    return angle
  
#2次元からの角度計算の定義
def calculate_2D_angle(a,b,c):
  a=np.array(a)#firtst
  b=np.array(b)#second
  c=np.array(c)#third
  radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
  angle=np.abs(radians*180/np.pi)

  if angle>180.0:
    angle=360-angle
  
  return angle

In [23]:
import mediapipe as mp
import cv2
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mesh_drawing_spec = mp_drawing.DrawingSpec(thickness=2, color=(0, 255, 0))#RGBで色指定
mark_drawing_spec = mp_drawing.DrawingSpec(thickness=3, circle_radius=3, color=(0, 0, 255))#RGBで色指定

img_path = fullpath

with mp_pose.Pose(
        min_detection_confidence=0.5,
        static_image_mode=True) as pose_detection:
    image = cv2.imread(img_path)
    image = cv2.resize(image, dsize=None, fx=0.3, fy=0.3)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    height = rgb_image.shape[0]
    width = rgb_image.shape[1]

    results = pose_detection.process(rgb_image)

    annotated_image = image.copy()
    if not results.pose_landmarks:
        print('not results')
    else:
   #各検出ポイントをarrayに変換
      rtshoulder=[results.pose_landmarks.landmark[12].x,results.pose_landmarks.landmark[12].y,results.pose_landmarks.landmark[12].z]
      rtelbow=[results.pose_landmarks.landmark[14].x,results.pose_landmarks.landmark[14].y,results.pose_landmarks.landmark[14].z]
      rtwrist=[results.pose_landmarks.landmark[16].x,results.pose_landmarks.landmark[16].y,results.pose_landmarks.landmark[16].z]
      rthip=[results.pose_landmarks.landmark[24].x,results.pose_landmarks.landmark[24].y,results.pose_landmarks.landmark[24].z]
      ltshoulder=[results.pose_landmarks.landmark[11].x,results.pose_landmarks.landmark[11].y,results.pose_landmarks.landmark[11].z]
  #3D座標から角度を計算 2Dか3Dか選ぶ
  #rtelbowAngleは右肩-右肘-右手関節
      rtelbowAngle=180-calculate_2D_angle(rtshoulder,rtelbow,rtwrist)
  #rtshoulderAngleは左肩-右肩-右肘
      rtshoulderAngle=calculate_2D_angle(ltshoulder,rtshoulder,rtelbow)
  #rtshoulderAbductionは右股関節-右肩-右肘
      rtshoulderAbduction=calculate_2D_angle(rthip,rtshoulder,rtelbow)
  #角度情報をテキストして画像に挿入
      cv2.putText(annotated_image, 'rtElbow'+str(round(rtelbowAngle,1)),
              (int(0.1*width),int(0.1*height)),
              cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,50),1,cv2.LINE_AA)

      cv2.putText(annotated_image, 'rtshoulderAbd'+str(round(rtshoulderAbduction,1)),
               (int(0.1*width),int(0.2*height)),    
              cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,50),1,cv2.LINE_AA)
      cv2.putText(annotated_image, 'rtshoulderAngle'+str(round(rtshoulderAngle,1)),
               (int(0.1*width),int(0.3*height)),
              cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,50),1,cv2.LINE_AA)
      mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mark_drawing_spec,
            connection_drawing_spec=mesh_drawing_spec
            )
      cv2.imwrite('result.jpg', annotated_image)
      print('right shoulder angle:',rtshoulderAngle)
      print('right Elbow:',rtelbowAngle)
      print('right shouler Abduction', rtshoulderAbduction)

right shoulder angle: 178.69184960987752
right Elbow: 6.220027692433092
right shouler Abduction 108.11592729569743


In [19]:
height

360

In [ ]:
rtshoulderAngle

In [ ]:
"""# 初期設定 holistic とPoseを用意する　検出感度は0.5＞＞変更可能
import mediapipe as mp
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.8)
pose= mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.8)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(color=[225,0,200],thickness=2, circle_radius=1) #color=[B,G,R]で指定,線の太さと〇の色も指定可能"""

In [ ]:
"""cap = cv2.VideoCapture(fullpath) #動画の読み込み
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #幅
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #高
frame_rate = int(cap.get(cv2.CAP_PROP_FPS)) #FPS
fourcc = cv2.VideoWriter_fourcc('m','p','4','v') #mp4出力
out = cv2.VideoWriter('out.mp4', fourcc, frame_rate, (width, height))""

# MediaPipeで静止画を処理

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob
import time
#変数の初期化
rtelbowAnglelist=[]
rtshoulderAnglelist=[]
rtshoulderAbductionlist=[]
timelist=[]
data_land = np.zeros((0,132))

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./images/*.png')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}
w = width             # 画像の横幅を取得
h = height            # 画像の縦幅を取得

for name, image in images.items():
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  if image is None:
      break
  results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  landmarks = results.pose_landmarks
  if landmarks is None:
      print(name)
      continue
   #各検出ポイントをarrayに変換
  rtshoulder=[results.pose_landmarks.landmark[12].x,results.pose_landmarks.landmark[12].y,results.pose_landmarks.landmark[12].z]
  rtelbow=[results.pose_landmarks.landmark[14].x,results.pose_landmarks.landmark[14].y,results.pose_landmarks.landmark[14].z]
  rtwrist=[results.pose_landmarks.landmark[16].x,results.pose_landmarks.landmark[16].y,results.pose_landmarks.landmark[16].z]
  rthip=[results.pose_landmarks.landmark[24].x,results.pose_landmarks.landmark[24].y,results.pose_landmarks.landmark[24].z]
  ltshoulder=[results.pose_landmarks.landmark[11].x,results.pose_landmarks.landmark[11].y,results.pose_landmarks.landmark[11].z]
  #3D座標から角度を計算 2Dか3Dか選ぶ
  #rtelbowAngleは右肩-右肘-右手関節
  rtelbowAngle=180-calculate_2D_angle(rtshoulder,rtelbow,rtwrist)
  #rtshoulderAngleは左肩-右肩-右肘
  rtshoulderAngle=calculate_2D_angle(ltshoulder,rtshoulder,rtelbow)
  #rtshoulderAbductionは右股関節-右肩-右肘
  rtshoulderAbduction=calculate_2D_angle(rthip,rtshoulder,rtelbow)
  #角度情報をテキストして画像に挿入
  cv2.putText(image, 'rtElbow'+str(round(rtelbowAngle,1)),
              (200,400),
              #tuple(np.multiply(rtelbow[:2],[w, h]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,50),2,cv2.LINE_AA)

  cv2.putText(image, 'rtshoulderAbd'+str(round(rtshoulderAbduction,1)),
              (200,350),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,50),2,cv2.LINE_AA)
  cv2.putText(image, 'rtshoulderAngle'+str(round(rtshoulderAngle,1)),
              (200,300),
              #tuple(np.multiply(rtshoulder[:2],[w, h]).astype(int)),
              cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,50),2,cv2.LINE_AA)

        
  rtelbowAnglelist.append(rtelbowAngle)
  rtshoulderAnglelist.append(rtshoulderAngle)  
  rtshoulderAbductionlist.append(rtshoulderAbduction)
  timelist.append(time.time())
  # Draw pose landmarks.
  annotated_image = image.copy()
  # Get Landmarks
  #landmarks=results.pose_landmarks.landmark
 
  """
  Holisticを使用しない場合はこの部分をコメントアウト
  
  mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=mp_holistic.FACEMESH_TESSELATION,
      landmark_drawing_spec=drawing_spec,
      connection_drawing_spec=drawing_spec)
  """
  mp_drawing.draw_landmarks(
        image=annotated_image, 
        landmark_list=results.pose_landmarks, 
        connections=mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=drawing_spec,
        connection_drawing_spec=drawing_spec)
  #get coordinate poseの33か所の3次元座標データと確信度(visibilisty)をすべて結合してCSVに変換
  data_land2 = np.zeros((1,3))
    
  for x in range (0,33):#pose_landmarksかpose_world_landmarksか選ぶ
      data1 = results.pose_landmarks.landmark[x].x
      data2 = results.pose_landmarks.landmark[x].y
      data3 = results.pose_landmarks.landmark[x].z
      data4 = results.pose_landmarks.landmark[x].visibility
      keydata = np.hstack((data1,data2,data3,data4)).reshape(1,-1)
      data_land2 = np.hstack((data_land2,keydata))

  data_land2 = data_land2[:,3:]
  data_land = np.vstack((data_land,data_land2))
  final_data= np.vstack((landmarklist,data_land))
  cv2.imwrite(name, annotated_image)  

#savetxtではデータの型が合わないと保存でき無いことがあるのでfmtで文字の型を指定
  np.savetxt('keypoint_results.csv',final_data,delimiter = ',',fmt = '%s')


# 角度のグラフ表示とデータのCSV保存

In [ ]:

from matplotlib import pyplot as plt
#combine Anglelist arrayにして水平結合
#timelistの開始数字を0に変更
np.array(timelist[0])
d=np.full(len(timelist),timelist[0])#すべての要素がtimelistの１番目の値でlen(timelist)の行列を作成
#timelistの時間を初期化（1番目の要素を0として行列の引き算
deltatime=np.array(timelist)-np.full(len(timelist),timelist[0])
a=np.array(rtelbowAnglelist)
b=np.array(rtshoulderAnglelist)
c=np.array(rtshoulderAbductionlist)
plt.plot(deltatime,a, label='elbow angle')
plt.plot(deltatime,b, label='shoulder angle')
plt.plot(deltatime,c, label='shoulder Abduction')
plt.title('Angle')
plt.legend()
plt.show() 
#角度データをAngle＿Results.CSVとして保存
name=['time','elbow_angle', 'shoulder_angle', 'shoulder_Abduction']
data=np.stack((deltatime,a,b,c)).T
data=np.vstack((name,data))
np.savetxt('Angle_Results.csv',data,delimiter = ',',fmt = '%s')

In [ ]:
#plotlyでグラフを可視化
import plotly
import plotly.graph_objs as go
# Google Colab. やJupyter Lab.でプロットするためには，以下を実行する．
import plotly.io as pio
pio.renderers.default = "colab"
plotly.__version__


In [ ]:
#Ea_roll_mean = a.rolling(10).mean() # 移動平均を求める

In [ ]:
fig = go.Figure() # 1 

fig.add_trace(    #2. グラフを追加・記述する
    go.Scatter(x = deltatime, y = a,
                  name='elbow angle')
)
fig.add_trace(
    go.Scatter(x= deltatime, y =b, name='shoulder angle'))
fig.add_trace(
    go.Scatter(x=deltatime, y= c, name='shoulder abduction')
)

fig.update_layout( #3. グラフの装飾、制限を追加(titleの表示、x軸の最大値など)
    title = 'Joint angles over time',
    xaxis_title = 'time',
    yaxis_title = 'angle',
    showlegend = True
)
fig.show() #4.グラフの描画

# 処理した画像をmp4動画に変換


In [ ]:
# 既に output.mp4 があれば削除
import os
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')

! ffmpeg -r 10 -i images/%6d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# --- mp4動画の再生 ---
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="85%" height="85%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

＊作成したmp4動画は、左側のファイル欄にある　./mediapipe/output.mp4 を右クリックするとダウンロードできます。